<a href="https://colab.research.google.com/github/Ozk18532/Contador/blob/main/Copy_of_Examen3_ZFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Examen 3: Algoritmo Colonia de hormiga**

# **1) Parámetros del problema y utilidades**
Pequeño módulo con los datos fijos del knapsack, parámetros por defecto del ACO y funciones simples de evaluación.

In [ ]:
# 💡 Cambios importantes respecto al documento ACO original:
# 1️⃣ Modularización del código por etapas (mejor estructura)
# 2️⃣ Construcción correcta de soluciones con vectores binarios
# 3️⃣ Inclusión de heurística η = ganancia/peso
# 4️⃣ Actualización dinámica de feromonas con evaporación + depósito
# 5️⃣ Control de semilla aleatoria para reproducibilidad
# 6️⃣ Registro de la mejor generación y vector solución
# 7️⃣ 30 corridas automáticas con métricas estadísticas
# 8️⃣ Salida organizada (tabla + resumen final)
# 9️⃣ Parámetros encapsulados en clase ACOParams
#
# Cada módulo del código explica con comentarios detallados
# qué se cambió, por qué se hizo y cómo mejora el algoritmo.
# ============================================================


# ============================================================
# MÓDULO 1: DATOS Y FUNCIONES DE UTILIDAD
# ------------------------------------------------------------
# ✏️ En el código original los pesos y ganancias estaban fijos,
# pero las funciones de evaluación no estaban separadas.
# Aquí modularizamos los datos, la función de aptitud (fitness),
# y el cálculo de la heurística η = ganancia/peso.
# ============================================================

# --- Módulo 1: Datos y utilidades ---

from dataclasses import dataclass
from typing import List, Tuple
import random, math, statistics as stats
import pandas as pd

# Datos del problema (15 ítems)
PESOS     = [70,73,77,80,82,87,90,94,98,106,110,113,115,118,120]
GANANCIAS = [135,139,149,150,156,163,173,184,192,201,210,214,221,229,240]
CAPACIDAD = 750
N = len(PESOS)

@dataclass
class ACOParams:
    generaciones: int = 100         # NumGen
    colonia: int = 100              # TamHormigas (ajuste solicitado)
    alpha: float = 1.0              # influencia de feromonas
    beta: float  = 2.0              # influencia de heurística (valor/peso)
    rho: float   = 0.50             # Tasa de evaporación
    q: float     = 1.0              # constante de depósito
    q0: float    = 0.10             # prob. de explotación (opcional)

def fitness(sol: List[int]) -> Tuple[int,int]:
    """Regresa (ganancia_total, peso_total). Asume solución factible."""
    peso = sum(w for w, b in zip(PESOS, sol) if b)
    valor = sum(v for v, b in zip(GANANCIAS, sol) if b)
    return valor, peso

def es_factible(sol: List[int]) -> bool:
    return sum(w for w, b in zip(PESOS, sol) if b) <= CAPACIDAD

# Heurística estática (mayor es mejor): valor/peso
ETA = [g/p for g,p in zip(GANANCIAS, PESOS)]


# **2) Construcción de una solución por hormiga**
Cada hormiga construye un vector binario. Se calcula, para cada ítem, una intensidad = (τᵢ^α)*(ηᵢ^β).

Se recorre en orden aleatorio y se incluye un ítem si cabe y rand < score/(score+1). Con esto combinamos feromonas y heurística sin romper la mochila (no se requiere “reparación” después).

In [ ]:
# ------------------------------------------------------------
# 🧩 En el código original, las hormigas no generaban soluciones
# individuales; se acumulaban pesos globales.
# 🔧 Ahora, cada hormiga construye su vector binario completo,
# asegurando factibilidad (sin exceder la capacidad).
# ============================================================
# --- Módulo 2: Construcción de soluciones ---

def construir_solucion(taus: List[float], params: ACOParams, rng: random.Random) -> List[int]:
    sol = [0]*N
    peso_actual = 0
    # recorrer ítems en orden aleatorio para favorecer diversidad
    indices = list(range(N))
    rng.shuffle(indices)

    for i in indices:
        if peso_actual + PESOS[i] > CAPACIDAD:
            continue  # no cabe
        # intensidad (τ^α * η^β)
        score = (taus[i] ** params.alpha) * (ETA[i] ** params.beta)
        # prob. de aceptación acotada y suave
        p_take = score / (score + 1.0)
        if rng.random() < p_take:
            sol[i] = 1
            peso_actual += PESOS[i]

    # Asegurar factibilidad (por diseño ya lo es)
    assert es_factible(sol)
    return sol


# **3) Actualización de feromonas**
Se evapora τ ← (1−ρ)τ y se deposita solo en los ítems de la mejor global (explotación estable) y en la mejor de la generación (exploración controlada).

In [ ]:
# ------------------------------------------------------------
# 🐜 Antes: se sumaban ganancias fijas a todas las feromonas.
# 🔧 Ahora: se evapora τ y se deposita solo sobre los ítems
# de la mejor hormiga de la iteración y la mejor global.
# Esto guía la exploración hacia combinaciones exitosas.
# ============================================================
# --- Módulo 3: Actualización de feromonas ---

def actualizar_feromonas(taus: List[float],
                         sol_global: List[int],
                         sol_iter: List[int],
                         valor_global: int,
                         valor_iter: int,
                         params: ACOParams):
    # Evaporación
    for i in range(N):
        taus[i] *= (1.0 - params.rho)
        if taus[i] < 1e-12:
            taus[i] = 1e-12  # evitar cero

    # Depósito proporcional al valor
    for i, bit in enumerate(sol_global):
        if bit:
            taus[i] += params.q * (valor_global)

    # Ligero refuerzo a la mejor de la iteración (suaviza la búsqueda)
    bonus = max(1.0, 0.10 * valor_iter)
    for i, bit in enumerate(sol_iter):
        if bit:
            taus[i] += bonus


# **4) Ejecución de una corrida de ACO**
Corre el algoritmo por generaciones, devuelve mejor valor, solución y generación donde apareció.

In [ ]:
 #------------------------------------------------------------
# 🧠 Corre una iteración completa del ACO.
# Guarda la mejor solución, vector y generación.
# ============================================================
# --- Módulo 4: Una corrida de ACO ---

def ejecutar_aco(params: ACOParams, seed: int = 42) -> Tuple[int, List[int], int]:
    rng = random.Random(seed)

    # feromonas iniciales homogéneas
    taus = [1.0]*N

    mejor_global_val, mejor_global_sol = -1, [0]*N
    gen_mejor_global = 0

    for gen in range(1, params.generaciones+1):
        mejor_iter_val, mejor_iter_sol = -1, None

        # Construcción de soluciones por colonia
        for _ in range(params.colonia):
            sol = construir_solucion(taus, params, rng)
            val, _ = fitness(sol)
            if val > mejor_iter_val:
                mejor_iter_val, mejor_iter_sol = val, sol

        # Actualizar mejor global
        if mejor_iter_val > mejor_global_val:
            mejor_global_val = mejor_iter_val
            mejor_global_sol = mejor_iter_sol[:]
            gen_mejor_global = gen

        # Actualizar feromonas con mejor global + mejor de iteración
        actualizar_feromonas(taus, mejor_global_sol, mejor_iter_sol,
                             mejor_global_val, mejor_iter_val, params)

        # Criterio de paro por óptimo conocido (opcional)
        if mejor_global_val >= 1458:
            break

    return mejor_global_val, mejor_global_sol, gen_mejor_global


# **5) Experimento con 30 corridas y tabla de resultados**
Ejecuta 30 veces con semillas controladas (42 + i).
Al final imprime peor, mejor, promedio y desviación estándar redondeados y te deja un DataFrame.

In [ ]:
# ------------------------------------------------------------
# 📊 Antes: solo se ejecutaba una corrida y se imprimía la mejor.
# 🔧 Ahora: se ejecutan 30 corridas con diferentes semillas y
# se calcula Peor, Mejor, Promedio y Desviación Estándar.
# ============================================================
# --- Módulo 5: 30 corridas y resumen ---

def correr_experimentos(n_corridas: int = 30,
                        base_seed: int = 42,
                        params: ACOParams = ACOParams()) -> pd.DataFrame:
    registros = []

    for i in range(1, n_corridas+1):
        val, sol, gen = ejecutar_aco(params, seed=base_seed + i)
        registros.append({"Corrida": i,
                          "Generación_mejor": gen,
                          "Mejor_valor": val})

    df = pd.DataFrame(registros)

    peor = int(df["Mejor_valor"].min())
    mejor = int(df["Mejor_valor"].max())
    promedio = round(float(df["Mejor_valor"].mean()), 3)
    desv = round(float(df["Mejor_valor"].std(ddof=1)), 3)  # ddof=1 para muestra

    # Resumen impreso
    print("=== Resumen 30 corridas (valores de la mejor solución de cada corrida) ===")
    print(f"Peor: {peor} | Mejor: {mejor} | Promedio: {promedio} | Desv. Est.: {desv}")

    # También agregamos una fila de resumen (opcional, útil para reporte)
    df_resumen = pd.DataFrame([{
        "Corrida": "RESUMEN",
        "Generación_mejor": "-",
        "Mejor_valor": f"Peor={peor}, Mejor={mejor}, Promedio={promedio}, Desv={desv}"
    }])

    return pd.concat([df, df_resumen], ignore_index=True)

# Ejecutar el experimento
params = ACOParams(
    generaciones=100,
    colonia=100,  # ajuste solicitado
    alpha=1.0,
    beta=2.0,
    rho=0.50,
    q=1.0,
    q0=0.10
)

df_resultados = correr_experimentos(n_corridas=30, base_seed=42, params=params)
df_resultados


=== Resumen 30 corridas (valores de la mejor solución de cada corrida) ===
Peor: 1450 | Mejor: 1458 | Promedio: 1456.733 | Desv. Est.: 2.227


,Corrida,Generación_mejor,Mejor_valor
0,1,17,1458
1,2,5,1458
2,3,8,1458
3,4,20,1458
4,5,1,1450
5,6,7,1453
6,7,5,1458
7,8,7,1458
8,9,7,1456
9,10,3,1458


# **6) Mostrar la mejor solución global de la última corrida**
Imprime el vector binario, su peso y su valor para documentar claramente el resultado.

In [ ]:
# ------------------------------------------------------------
# 🧾 Imprime el vector binario, peso, valor y generación donde
# se obtuvo la mejor solución (para documentar el resultado).
# ============================================================
# --- Módulo 6: Inspección de la última corrida ---

best_val, best_sol, best_gen = ejecutar_aco(params, seed=42+30)  # misma semilla que la última
val, peso = fitness(best_sol)

print("Mejor solución de la corrida 30")
print("Vector:", best_sol)
print(f"Valor: {val} | Peso: {peso} | Generación donde apareció: {best_gen}")


Mejor solución de la corrida 30
Vector: [1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1]
Valor: 1458 | Peso: 749 | Generación donde apareció: 5
